In [1]:
import arviz as az
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import seaborn.objects as so
import numpy as np
import pandas as pd
import polars as pl
import pybaseball
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

RANDOM_SEED = 504




In [4]:
import psycopg2
import os
import pandas as pd
from sqlalchemy import create_engine
def get_connection():
    conn = psycopg2.connect(
        dbname="statcast",
        user="postgres",
        password="jamin",
        host="localhost",
        port="5432"
    )
    return conn

In [15]:
#from retrieve_sql_module import get_events_data
#years = list(range(2021,2025))
events_data = get_events_data("2024")

C:\Users\jamin\AppData\Local\Temp\ipykernel_17636\3933921856.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(query,conn)


In [16]:
events_data

,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,...,field_error_n,sac_fly_n,n_pitches,total_events,single_rate,double_rate,triple_rate,home_run_rate,strikeout_rate,field_out_rate
0,KC,2024-10-30,77.5,-1.11,5.65,"Buehler, Walker",657077,621111,strikeout,swinging_strike_blocked,...,5,11,2605,684,0.141813,0.043860,0.001462,0.141813,0.144737,0.543860
1,KC,2024-10-30,77.4,-1.23,5.78,"Buehler, Walker",669224,621111,strikeout,swinging_strike,...,2,7,1894,481,0.112266,0.041580,0.002079,0.112266,0.232848,0.438669
2,KC,2024-10-30,77.6,-1.08,5.75,"Buehler, Walker",683011,621111,field_out,hit_into_play,...,5,3,3101,754,0.159151,0.039788,0.009284,0.159151,0.225464,0.468170
3,ST,2024-10-30,79.4,-1.48,5.81,"Leiter Jr., Mark",669257,643410,field_out,hit_into_play,...,3,9,2568,608,0.125000,0.041118,0.003289,0.125000,0.190789,0.481908
4,CU,2024-10-30,75.5,-1.14,6.05,"Leiter Jr., Mark",669242,643410,strikeout,swinging_strike,...,2,3,808,220,0.154545,0.045455,0.004545,0.154545,0.177273,0.490909
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
192067,FF,2024-03-20,96.6,-1.69,6.17,"Glasnow, Tyler",665487,607192,field_out,hit_into_play,...,0,1,1669,471,0.150743,0.050955,0.002123,0.150743,0.208068,0.439490
192068,FF,2024-03-20,96.9,-1.74,6.05,"Glasnow, Tyler",593428,607192,field_out,hit_into_play,...,2,6,1922,490,0.179592,0.034694,0.002041,0.179592,0.167347,0.516327
192069,FF,2024-03-20,94.5,-1.91,5.67,"Darvish, Yu",669257,506433,field_out,hit_into_play,...,3,9,2568,608,0.125000,0.041118,0.003289,0.125000,0.190789,0.481908
192070,SI,2024-03-20,95.4,-2.11,5.61,"Darvish, Yu",518692,506433,strikeout,called_strike,...,3,8,2759,690,0.147826,0.052174,0.004348,0.147826,0.159420,0.466667


In [6]:
fangraphs_batting_stats_2024 = pybaseball.batting_stats(2024, qual=5)

In [5]:
from retrieve_sql_module import upload_to_sql


D:\jamin\baseball-projects\projections\hitting\scripts\retrieve_sql_module.py:37: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  pbp_data = pd.concat(pbp_data_list, ignore_index=True)


In [8]:
upload_to_sql(fangraphs_batting_stats_2024, "fangraphs_batting_stats_2024")

DataFrame       IDfg  Season              Name   Team  Age    G   AB   PA    H   1B  \
3    15640    2024       Aaron Judge    NYY   32  158  559  704  180   85   
12   25764    2024    Bobby Witt Jr.    KCR   24  161  636  709  211  123   
7    19755    2024     Shohei Ohtani    LAD   29  159  636  731  197   98   
8    20123    2024         Juan Soto    NYY   25  157  576  713  166   90   
28   26289    2024  Gunnar Henderson    BAL   23  159  630  719  177  102   
..     ...     ...               ...    ...  ...  ...  ...  ...  ...  ...   
588   6887    2024  Martin Maldonado    CHW   37   48  135  147   16    9   
600  15676    2024        Jose Abreu    HOU   37   35  113  120   14   10   
492  20543    2024  Elehuris Montero    COL   25   67  224  247   46   32   
542  12155    2024     Eddie Rosario  - - -   32   91  297  319   52   30   
558  11615    2024     Brandon Drury    LAA   31   97  325  360   55   44   

     ...  maxEV  HardHit  HardHit%  Events  CStr%   CSW%    xBA  

In [14]:
from retrieve_sql_module import upload_to_sql

upload_to_sql(pbp_data, "statcast_all")

DataFrame         pitch_type  game_date  release_speed  release_pos_x  release_pos_z  \
0               FF 2021-10-03           92.3           1.40           6.80   
1               SL 2021-10-03           80.6           1.60           6.64   
2               CU 2021-10-03           75.5           1.46           6.88   
3               CU 2021-10-03           75.0           1.53           6.83   
4               FF 2021-10-03           91.2           1.49           6.66   
...            ...        ...            ...            ...            ...   
2891004         FF 2024-03-20           95.7          -2.13           5.77   
2891005         SI 2024-03-20           93.9          -2.17           5.70   
2891006         SI 2024-03-20           93.4          -2.00           5.70   
2891007         FF 2024-03-20           92.6          -1.94           5.83   
2891008         FF 2024-03-20           94.5          -1.79           5.83   

         player_name  batter  pitcher     events    d